# Run Agent
- Implement a workflow to write a daily AI newsletter
- see README.md for details


In [1]:
import os
import yaml
import dotenv
import logging
import json
import yaml
from datetime import datetime
import time
import random
import glob
import pickle
import sqlite3

from pathlib import Path

import asyncio
import nest_asyncio

import pydantic
from pydantic import BaseModel, Field, RootModel
from typing import Dict, TypedDict, Type, List, Optional, Any, Iterable, Text
from dataclasses import dataclass, field
from enum import Enum

import numpy as np
import pandas as pd

import pandas as pd
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
import hdbscan

import openai
from openai import AsyncOpenAI

import agents
from agents.exceptions import InputGuardrailTripwireTriggered
from agents import (Agent, Runner, Tool, OpenAIResponsesModel, 
                    ModelSettings, FunctionTool, InputGuardrail, GuardrailFunctionOutput,
                    SQLiteSession, set_default_openai_api, set_default_openai_client
                   )


import tenacity
from tenacity import retry, stop_after_attempt, wait_exponential, retry_if_exception_type

from IPython.display import HTML, Image, Markdown, display

from log_handler import SQLiteLogHandler, setup_sqlite_logging, sanitize_error_for_logging
from config import LOGDB
from llm import LLMagent, LangfuseClient  # methods to apply prompts async to large batches
from db import Url 

from fetch import Fetcher # fetch news urls
from newsletter_state import NewsletterAgentState, StepStatus
from news_agent import NewsletterAgent


In [2]:
print(f"OpenAI:            {openai.__version__}")
print(f"OpenAI Agents SDK  {agents.__version__}")
print(f"Pydantic           {pydantic.__version__}")


OpenAI:            1.109.0
OpenAI Agents SDK  0.3.1
Pydantic           2.11.9


In [3]:
dotenv.load_dotenv()

# to run async in jupyter notebook
nest_asyncio.apply()

# verbose OpenAI console logging if something doesn't work
# logging.basicConfig(level=logging.DEBUG)
# openai_logger = logging.getLogger("openai")
# openai_logger.setLevel(logging.DEBUG)


In [4]:
# modules create a default logger, or we can pass this logger

def setup_logging(session_id: str = "default", db_path: str = "agent_logs.db") -> logging.Logger:
    """Set up logging to console and SQLite database."""

    # Create logger
    logging.basicConfig(level=logging.INFO)

    logger = logging.getLogger(f"NewsletterAgent.{session_id}")
    logger.setLevel(logging.INFO)

    # Clear any existing handlers
    logger.handlers.clear()

    # Console handler
    console_handler = logging.StreamHandler()
    console_handler.setLevel(logging.INFO)
    console_formatter = logging.Formatter(
        '%(asctime)s | %(name)s | %(levelname)s | %(message)s',
        datefmt='%H:%M:%S'
    )
    console_handler.setFormatter(console_formatter)

    # SQLite handler
    sqlite_handler = SQLiteLogHandler(db_path)
    sqlite_handler.setLevel(logging.INFO)
    sqlite_formatter = logging.Formatter('%(message)s')
    sqlite_handler.setFormatter(sqlite_formatter)

    # Add handlers to logger
    logger.addHandler(console_handler)
    logger.addHandler(sqlite_handler)

    # Prevent propagation to root logger
    logger.propagate = False

    return logger

logger = setup_logging("newsletter_agent", "test_logs.db")

# Log some test messages
logger.info("Test info message", extra={
    'step_name': 'test_step',
    'agent_session': 'demo_session'
})

logger.warning("Test warning message", extra={
    'step_name': 'test_step',
    'agent_session': 'demo_session'
})

logger.error("Test error message", extra={
    'step_name': 'error_step',
    'agent_session': 'demo_session'
})

sanitize_error_for_logging("log with some bad stuff for the filter: sk-proj-123456789012345678901234567890123456789012345678")

14:02:57 | NewsletterAgent.newsletter_agent | INFO | Test info message
14:02:57 | NewsletterAgent.newsletter_agent | WARNING | Test warning message
14:02:57 | NewsletterAgent.newsletter_agent | ERROR | Test error message


'log with some bad stuff for the filter: [API_KEY_REDACTED]'

# Run Agent Worfklow

In [5]:
print("🚀 Creating NewsletterAgent...")

do_download=True
process_since=None
# process_since='2025-10-05 18:30:00'

try:
    # set up state
    session_id = 'test_newsletter_20251012081135108761'
    step_name = 'step_08_draft_sections'
#     del session_id, step_name
except Exception as e:
    print(e)

api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("OPENAI_API_KEY environment variable not set")

# Set up OpenAI client for the agents SDK
set_default_openai_client(AsyncOpenAI(api_key=api_key))

# Create agent with persistent state
if 'session_id' in vars():
    # load state from db for session_id and state
    print("session_id is defined")
    print(session_id)
    state = NewsletterAgentState(session_id=session_id, 
                                 db_path="newsletter_agent.db", 
                                 do_download=do_download,
                                 process_since=process_since,
                                 verbose=False
                                )
    state = state.load_from_db(step_name)
    agent = NewsletterAgent(session_id=session_id, state=state, verbose=True, timeout=30)    
else:
    # create new session
    print("session_id is not defined")
    timestamp = datetime.now().strftime("%Y%m%d%H%M%S%f")    
    session_id = f"test_newsletter_{timestamp}"
    print(session_id)
    state = NewsletterAgentState(session_id=session_id, 
                                 db_path="newsletter_agent.db",
                                 do_download=do_download,
                                 process_since=process_since,
                                 verbose=False
                                ) 
    agent = NewsletterAgent(session_id=session_id, state=state, verbose=False, timeout=30)
    state.serialize_to_db("initialize")

14:02:57 | NewsletterAgent.test_newsletter_20251012081135108761 | INFO | Using provided state with 42 articles


🚀 Creating NewsletterAgent...
session_id is defined
test_newsletter_20251012081135108761
Initialized NewsletterAgent with persistent state and 9-step workflow
Session ID: test_newsletter_20251012081135108761


In [6]:
agent.state.get_status()


{'headlines': {'total': 42},
 'sources': {'config_file': 'sources.yaml', 'loaded_sources': 0},
 'topics': {'cluster_topics': 0, 'topics': []},
 'workflow': {'current_step': 'step_09_finalize_newsletter',
  'workflow_complete': False,
  'workflow_status': 'started',
  'workflow_status_message': 'Drafted 30 stories across 8 sections',
  'progress_percentage': 88.88888888888889,
  'max_edits': 2,
  'concurrency': 16,
  'do_download': True,
  'process_since': None},
 'processing': {'topic_clusters': 0,
  'newsletter_sections': 30,
  'final_newsletter_length': 0}}

In [7]:
# User prompt to run workflow
user_prompt = "Show the workflow status"

print(f"\n📝 User prompt: '{user_prompt}'")
print("=" * 80)

# Run the agent with persistent state
start_time = time.time()
result = await agent.run_step(user_prompt)
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)


📝 User prompt: 'Show the workflow status'


08:11:37 | NewsletterAgent.test_newsletter_20251012081135108761 | INFO | Starting check_workflow_status
08:11:37 | NewsletterAgent.test_newsletter_20251012081135108761 | INFO | Completed check_workflow_status


⏱️  Total execution time: 6.72s
📊 Final result:
Current workflow status:

- Progress: 0.0% (0/9 complete)
- Status: 0 complete, 0 started, 0 failed, 9 not started
- Next step: Step 1 — Fetch URLs

Step-by-step status:
- Step 1: Fetch Urls — not_started
- Step 2: Filter Urls — not_started
- Step 3: Download Articles — not_started
- Step 4: Extract Summaries — not_started
- Step 5: Rate Articles — not_started
- Step 6: Cluster By Topic — not_started
- Step 7: Select Sections — not_started
- Step 8: Draft Sections — not_started
- Step 9: Finalize Newsletter — not_started

Would you like me to start Step 1 (gather URLs) or resume/execute a specific step?


In [8]:
# User prompt to run a workflow step
user_prompt = "Run step 1, fetch urls"

print(f"\n📝 User prompt: '{user_prompt}'")
print("=" * 80)

# Run the agent with persistent state
start_time = time.time()
result = await agent.run_step(user_prompt)
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)



📝 User prompt: 'Run step 1, fetch urls'


08:11:44 | NewsletterAgent.test_newsletter_20251012081135108761 | INFO | Starting check_workflow_status
08:11:44 | NewsletterAgent.test_newsletter_20251012081135108761 | INFO | Completed check_workflow_status
08:11:45 | NewsletterAgent.test_newsletter_20251012081135108761 | INFO | Starting Step 1: Gather URLs
08:11:45 | NewsletterAgent.test_newsletter_20251012081135108761 | INFO | Cleaning download/sources: 
08:11:45 | NewsletterAgent.test_newsletter_20251012081135108761 | INFO | Successfully cleaned download/sources
08:11:45 | NewsletterAgent.test_newsletter_20251012081135108761 | INFO | Cleaning download/text: 
08:11:45 | NewsletterAgent.test_newsletter_20251012081135108761 | INFO | Successfully cleaned download/text
08:11:45 | NewsletterAgent.test_newsletter_20251012081135108761 | INFO | Cleaning download/html: 
08:11:46 | NewsletterAgent.test_newsletter_20251012081135108761 | INFO | Successfully cleaned download/html
2025-10-12 08:11:46,007 - fetcher_5016884752 - INFO - [fetcher_in

▶ Starting Step 1: step_01_fetch_urls


2025-10-12 08:11:46,313 - fetcher_5016884752 - INFO - [fetch_rss] RSS fetch successful for HackerNoon: 50 articles
2025-10-12 08:11:46,314 - fetcher_5016884752 - INFO - [fetch_html] Fetching HTML from Techmeme: https://www.techmeme.com/river
2025-10-12 08:11:46,346 - fetcher_5016884752 - INFO - [fetch_rss] RSS fetch successful for Ars Technica: 20 articles
2025-10-12 08:11:46,348 - fetcher_5016884752 - INFO - [fetch_rss] Fetching RSS from The Register: https://www.theregister.com/software/ai_ml/headlines.atom
2025-10-12 08:11:46,428 - fetcher_5016884752 - INFO - [fetch_rss] RSS fetch successful for Hacker News: 30 articles
2025-10-12 08:11:46,428 - fetcher_5016884752 - INFO - [fetch_html] Fetching HTML from The Verge: https://www.theverge.com/ai-artificial-intelligence
2025-10-12 08:11:46,458 - fetcher_5016884752 - INFO - [fetch_rss] RSS fetch successful for The Register: 50 articles
2025-10-12 08:11:46,458 - fetcher_5016884752 - INFO - [fetch_html] Fetching HTML from VentureBeat: http

2025-10-12 08:11:47,939 - fetcher_5016884752 - INFO - Downloading https://feedly.com/i/aiFeeds?options=eyJsYXllcnMiOlt7InBhcnRzIjpbeyJpZCI6Im5scC9mL3RvcGljLzMwMDAifV0sInNhbGllbmNlIjoibWVudGlvbiIsInNlYXJjaEhpbnQiOiJ0ZWNobm9sb2d5IiwidHlwZSI6Im1hdGNoZXMifV0sImJ1bmRsZXMiOnsiYWxsRmVlZGx5TW9kZSI6W3sidHlwZSI6InN0cmVhbSIsImlkIjoidXNlci82MmVlYmI5Zi03MTUxLTRmOWEtYThjNy05YTU3YjgyMDUzMDgvY2F0ZWdvcnkvZ2xvYmFsLmFsbCJ9LHsidHlwZSI6InB1YmxpY2F0aW9uQnVja2V0IiwiaWQiOiJkaXNjb3Zlcnk6YWxsLXRvcGljcyIsInRpZXIiOiJ0aWVyMSJ9XSwiY3VzdG9tTW9kZSI6W3sidHlwZSI6InN0cmVhbSIsImlkIjoidXNlci82MmVlYmI5Zi03MTUxLTRmOWEtYThjNy05YTU3YjgyMDUzMDgvY2F0ZWdvcnkvZ2xvYmFsLmFsbCJ9XSwiYm9hcmRNb2RlIjpbeyJ0eXBlIjoic3RyZWFtIiwiaWQiOiJ1c2VyLzYyZWViYjlmLTcxNTEtNGY5YS1hOGM3LTlhNTdiODIwNTMwOC90YWcvZ2xvYmFsLmFsbCJ9XSwiYW5ub3RhdGVkTW9kZSI6W3sidHlwZSI6InN0cmVhbSIsImlkIjoidXNlci82MmVlYmI5Zi03MTUxLTRmOWEtYThjNy05YTU3YjgyMDUzMDgvdGFnL2dsb2JhbC5hbm5vdGF0ZWQifV19LCJyZWZpbmVNb2RlIjoiY3VzdG9tTW9kZSIsImxhbmd1YWdlcyI6WyJlbiJdfQ
2025-10-12 08:11:47,940 - 

...

2025-10-12 08:12:13,290 - fetcher_5016884752 - DEBUG - Converted to UTC: 2025-10-12 13:12:13+00:00
2025-10-12 08:12:13,290 - fetcher_5016884752 - DEBUG - Formatted last_updated: 2025-10-12T13:12:13Z
2025-10-12 08:12:13,290 - fetcher_5016884752 - INFO - Saving HTML to download/sources/WSJ.html
2025-10-12 08:12:13,299 - fetcher_5016884752 - INFO - [fetch_html] Parsing HTML file: download/sources/WSJ.html
2025-10-12 08:12:13,326 - fetcher_5016884752 - INFO - [fetch_html] Parsed HTML file: download/sources/WSJ.html
2025-10-12 08:12:13,327 - fetcher_5016884752 - INFO - [fetch_html] HTML fetch successful for WSJ: 26 articles
2025-10-12 08:12:13,752 - fetcher_5016884752 - INFO - Scrolling Reddit (4/5)
2025-10-12 08:12:15,774 - fetcher_5016884752 - INFO - Scrolling Reddit (5/5)
2025-10-12 08:12:17,705 - fetcher_5016884752 - DEBUG - Found last updated time from document.lastModified: 10/12/2025 13:12:17
2025-10-12 08:12:17,705 - fetcher_5016884752 - DEBUG - Attempting to parse last_updated: '10

,source,url
0,Ars Technica,20
1,Bloomberg,31
2,Business Insider,15
3,FT,46
4,Feedly AI,69
5,Hacker News,28
6,HackerNoon,50
7,New York Times,21
8,NewsAPI,43
9,Reddit,48


,source,title,url,published,rss_summary,id
0,Ars Technica,OpenAI will stop saving most ChatGPT users’ de...,https://arstechnica.com/tech-policy/2025/10/op...,"Fri, 10 Oct 2025 14:58:29 +0000",Court ends controversial order forcing OpenAI ...,0
1,Ars Technica,“Like putting on glasses for the first time”—h...,https://arstechnica.com/science/2025/10/like-p...,"Fri, 10 Oct 2025 11:00:24 +0000",AI is “comically good” at detecting small eart...,1
2,Ars Technica,AI models can acquire backdoors from surprisin...,https://arstechnica.com/ai/2025/10/ai-models-c...,"Thu, 09 Oct 2025 22:03:21 +0000","Anthropic study suggests ""poison"" training att...",2
3,Ars Technica,Bank of England warns AI stock bubble rivals 2...,https://arstechnica.com/ai/2025/10/bank-of-eng...,"Wed, 08 Oct 2025 21:18:30 +0000",Central bank says market concentration hasn't ...,3
4,Ars Technica,Vandals deface ads for AI necklaces that liste...,https://arstechnica.com/tech-policy/2025/10/va...,"Wed, 08 Oct 2025 16:07:29 +0000",Critics attacked subway ads to defend human fr...,4
...,...,...,...,...,...,...
697,NewsAPI,OpenAI's dominance is unlike anything Silicon ...,https://www.cnbc.com/2025/10/11/open-ai-silico...,2025-10-11T12:00:01Z,NaN,697
698,NewsAPI,Revolutionary AI-Powered Marketing Platform La...,https://rivercountry.newschannelnebraska.com/s...,2025-10-11T09:44:41Z,NaN,698
699,NewsAPI,Revolutionary AI-Powered Marketing Platform La...,https://panhandle.newschannelnebraska.com/stor...,2025-10-11T10:47:55Z,NaN,699
700,NewsAPI,Fact check: Trump’s ‘$17 trillion’ investment ...,https://www.cnn.com/2025/10/11/politics/fact-c...,2025-10-11T11:44:47Z,NaN,700


08:12:22 | NewsletterAgent.test_newsletter_20251012081135108761 | INFO | Completed Step 1: Gathered 807 articles
08:12:25 | NewsletterAgent.test_newsletter_20251012081135108761 | INFO | Starting check_workflow_status
08:12:25 | NewsletterAgent.test_newsletter_20251012081135108761 | INFO | Completed check_workflow_status


⏱️  Total execution time: 47.21s
📊 Final result:
Step 1 completed.

Summary:
- Fetch URLs completed successfully.
- Gathered 807 articles from 17 RSS sources; 702 articles stored in persistent state.
- Workflow progress: 1/9 complete (11.1%).
- Next step: Step 2 — Filter URLs (AI-related content)

Would you like me to proceed with Step 2 now?


In [9]:
pd.DataFrame(state.headline_data) 


,source,title,url,published,rss_summary,id
0,Ars Technica,OpenAI will stop saving most ChatGPT users’ de...,https://arstechnica.com/tech-policy/2025/10/op...,"Fri, 10 Oct 2025 14:58:29 +0000",Court ends controversial order forcing OpenAI ...,0
1,Ars Technica,“Like putting on glasses for the first time”—h...,https://arstechnica.com/science/2025/10/like-p...,"Fri, 10 Oct 2025 11:00:24 +0000",AI is “comically good” at detecting small eart...,1
2,Ars Technica,AI models can acquire backdoors from surprisin...,https://arstechnica.com/ai/2025/10/ai-models-c...,"Thu, 09 Oct 2025 22:03:21 +0000","Anthropic study suggests ""poison"" training att...",2
3,Ars Technica,Bank of England warns AI stock bubble rivals 2...,https://arstechnica.com/ai/2025/10/bank-of-eng...,"Wed, 08 Oct 2025 21:18:30 +0000",Central bank says market concentration hasn't ...,3
4,Ars Technica,Vandals deface ads for AI necklaces that liste...,https://arstechnica.com/tech-policy/2025/10/va...,"Wed, 08 Oct 2025 16:07:29 +0000",Critics attacked subway ads to defend human fr...,4
...,...,...,...,...,...,...
697,NewsAPI,OpenAI's dominance is unlike anything Silicon ...,https://www.cnbc.com/2025/10/11/open-ai-silico...,2025-10-11T12:00:01Z,NaN,697
698,NewsAPI,Revolutionary AI-Powered Marketing Platform La...,https://rivercountry.newschannelnebraska.com/s...,2025-10-11T09:44:41Z,NaN,698
699,NewsAPI,Revolutionary AI-Powered Marketing Platform La...,https://panhandle.newschannelnebraska.com/stor...,2025-10-11T10:47:55Z,NaN,699
700,NewsAPI,Fact check: Trump’s ‘$17 trillion’ investment ...,https://www.cnn.com/2025/10/11/politics/fact-c...,2025-10-11T11:44:47Z,NaN,700


In [10]:
countdf = pd.DataFrame(state.headline_data) \
    .groupby("source") \
    .count()[["id"]] \
    .reset_index() \
    .rename(columns={'id': 'count'}) \
    .sort_values("count", ascending=False)
countdf 


,source,count
10,Techmeme,179
4,Feedly AI,69
12,The Register,50
6,HackerNoon,50
9,Reddit,48
3,FT,46
8,NewsAPI,43
1,Bloomberg,31
13,The Verge,29
5,Hacker News,28


In [11]:
# Run tool directly without LLM processing an input prompt or results
# user_prompt = "Run step 2, filter urls"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

# Run the agent with persistent state
start_time = time.time()
result = await agent.run_tool_direct("filter_urls")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)


08:12:29 | NewsletterAgent.test_newsletter_20251012081135108761 | INFO | Starting Step 2: Filter URLs
08:12:29 | NewsletterAgent.test_newsletter_20251012081135108761 | INFO | 🔍 Filtering 702 headlines...
08:12:29 | NewsletterAgent.test_newsletter_20251012081135108761 | INFO | 🔄 Checking for duplicates (all urls without date restrictions)
08:12:29 | NewsletterAgent.test_newsletter_20251012081135108761 | INFO | 🔍 Filtering 702 articles for dupes.


▶ Starting Step 2: step_02_filter_urls
checking https://arstechnica.com/tech-policy/2025/10/openai-no-longer-forced-to-save-deleted-chats-but-some-users-still-affected/
found url, no cutoff set - treating as duplicate
checking https://arstechnica.com/science/2025/10/like-putting-on-glasses-for-the-first-time-how-ai-improves-earthquake-detection/
found url, no cutoff set - treating as duplicate
checking https://arstechnica.com/ai/2025/10/ai-models-can-acquire-backdoors-from-surprisingly-few-malicious-documents/
found url, no cutoff set - treating as duplicate
checking https://arstechnica.com/ai/2025/10/bank-of-england-warns-ai-stock-bubble-rivals-2000-dotcom-peak/
found url, no cutoff set - treating as duplicate
checking https://arstechnica.com/tech-policy/2025/10/vandals-deface-ads-for-ai-necklaces-that-listen-to-all-your-conversations/
found url, no cutoff set - treating as duplicate
checking https://arstechnica.com/ai/2025/10/insurers-balk-at-paying-out-huge-settlements-for-claims-ag

found url, no cutoff set - treating as duplicate
checking https://www.nytimes.com/2025/10/09/technology/roger-ver-bitcoin-justice-department-deal.html
found url, no cutoff set - treating as duplicate
checking https://www.nytimes.com/2025/10/09/business/new-jersey-electricity-prices-governor-election.html
found url, no cutoff set - treating as duplicate
checking https://www.nytimes.com/2025/10/08/science/nsf-spacecraft-technology.html
found url, no cutoff set - treating as duplicate
checking https://www.nytimes.com/2025/10/08/climate/trump-climate-science-funding.html
found url, no cutoff set - treating as duplicate
checking https://www.nytimes.com/2025/10/08/science/diamonds-computer-chips-ai.html
found url, no cutoff set - treating as duplicate
checking https://www.nytimes.com/2025/10/07/style/friend-ai-subway-ads-new-york.html
found url, no cutoff set - treating as duplicate
checking https://www.nytimes.com/2025/10/07/business/tesla-cheaper-model-y-3.html
found url, no cutoff set - t

08:12:29 | NewsletterAgent.test_newsletter_20251012081135108761 | INFO | URL deduplication: 531 duplicates filtered, 171 new URLs remain
08:12:29 | NewsletterAgent.test_newsletter_20251012081135108761 | INFO | 🔍 Filtering 702 headlines for AI relevance using LLM...
INFO:llm:Initialized LangfuseClient


checking https://www.reddit.com/r/ProgrammerHumor/comments/1o3vqub/weveallbeenthere/
found url, no cutoff set - treating as duplicate
checking https://v.redd.it/7lsdqsn7ckuf1
url not found - inserting new URL
checking https://www.reddit.com/r/aivideo/comments/1o49e8q/naruto_live_action_cast/
found url, no cutoff set - treating as duplicate
checking https://www.reddit.com/r/ProgrammerHumor/comments/1o41owb/somethingwewillneverlearn/
found url, no cutoff set - treating as duplicate
checking https://www.telegraph.co.uk/us/news/2025/10/10/barron-trump-tipped-top-tiktok-job/
url not found - inserting new URL
checking https://www.reddit.com/r/technology/comments/1o4kf83/us_presidents_son_barron_tipped_for_top_tiktok_job/
url not found - inserting new URL
checking https://www.cnn.com/2025/10/03/tech/tiktok-pushed-sexually-explicit-search-terms-to-13-year-olds-report
url not found - inserting new URL
checking https://www.reddit.com/r/technology/comments/1o45bxa/tiktok_searches_pushed_13yearold

INFO:llm:Successfully retrieved prompt 'newsagent/filter_urls' from Langfuse
INFO:llm:Parsed prompt 'newsagent/filter_urls': model=gpt-4.1-mini, system_len=459, user_len=954
08:12:29 | NewsletterAgent.test_newsletter_20251012081135108761 | INFO | Processing 7 chunks with concurrency 16


concurrency:  16


08:12:44 | NewsletterAgent.test_newsletter_20251012081135108761 | INFO | Completed Step 2: 60 AI-related articles, 531 duplicates removed


⏱️  Total execution time: 15.29s
📊 Final result:
✅ Step 2 step_02_filter_urls completed successfully! Removed 531 duplicate URLs, classified 171 new articles, found 60 AI-related.


In [12]:
# User prompt to run workflow
# user_prompt = "Run step 3, download full articles"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

# Run the agent with persistent state
start_time = time.time()
result = await agent.run_tool_direct("download_articles")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)

08:12:44 | NewsletterAgent.test_newsletter_20251012081135108761 | INFO | Starting Step 3: Download Articles
08:12:44 | NewsletterAgent.test_newsletter_20251012081135108761 | INFO | Starting concurrent scraping of 60 AI-related articles


▶ Starting Step 3: step_03_download_articles


08:12:44 | NewsletterAgent.test_newsletter_20251012081135108761 | INFO | Launching browser for 60 URLs with 16 concurrent workers
08:12:44 | NewsletterAgent.test_newsletter_20251012081135108761 | INFO | Worker 0 fetching 1 of 60 https://www.together.ai/blog/adaptive-learning-speculator-system-atlas
08:12:44 | NewsletterAgent.test_newsletter_20251012081135108761 | INFO | scrape_url(https://www.together.ai/blog/adaptive-learning-speculator-system-atlas)
08:12:44 | NewsletterAgent.test_newsletter_20251012081135108761 | INFO | scraping https://www.together.ai/blog/adaptive-learning-speculator-system-atlas to download/html
08:12:44 | NewsletterAgent.test_newsletter_20251012081135108761 | INFO | Downloading https://www.together.ai/blog/adaptive-learning-speculator-system-atlas
08:12:44 | NewsletterAgent.test_newsletter_20251012081135108761 | INFO | Worker 1 fetching 2 of 60 https://www.makeuseof.com/devils-advocate-prompt-chatgpt/
08:12:44 | NewsletterAgent.test_newsletter_202510120811351087

08:12:44 | NewsletterAgent.test_newsletter_20251012081135108761 | INFO | Worker 11 fetching 11 of 60 https://arxiv.org/abs/2505.00749
08:12:44 | NewsletterAgent.test_newsletter_20251012081135108761 | INFO | scrape_url(https://arxiv.org/abs/2505.00749)
08:12:44 | NewsletterAgent.test_newsletter_20251012081135108761 | INFO | scraping https://arxiv.org/abs/2505.00749 to download/html
08:12:44 | NewsletterAgent.test_newsletter_20251012081135108761 | INFO | Downloading https://arxiv.org/abs/2505.00749
08:12:44 | NewsletterAgent.test_newsletter_20251012081135108761 | INFO | Worker 12 fetching 12 of 60 https://www.fool.com/investing/2025/10/11/this-dividend-king-could-surge-75-by-2030-thanks-t/
08:12:44 | NewsletterAgent.test_newsletter_20251012081135108761 | INFO | Rate limiting domain www.fool.com, will retry later (need to wait 2.0s)
08:12:44 | NewsletterAgent.test_newsletter_20251012081135108761 | INFO | Worker 12 re-queued rate-limited URL: https://www.fool.com/investing/2025/10/11/this-

08:13:16 | NewsletterAgent.test_newsletter_20251012081135108761 | INFO | Worker 15 fetching 19 of 60 https://www.chinatalk.media/p/why-america-builds-ai-girlfriends
08:13:16 | NewsletterAgent.test_newsletter_20251012081135108761 | INFO | scrape_url(https://www.chinatalk.media/p/why-america-builds-ai-girlfriends)
08:13:16 | NewsletterAgent.test_newsletter_20251012081135108761 | INFO | scraping https://www.chinatalk.media/p/why-america-builds-ai-girlfriends to download/html
08:13:16 | NewsletterAgent.test_newsletter_20251012081135108761 | INFO | Downloading https://www.chinatalk.media/p/why-america-builds-ai-girlfriends
08:13:20 | NewsletterAgent.test_newsletter_20251012081135108761 | INFO | Response: 200
08:13:20 | NewsletterAgent.test_newsletter_20251012081135108761 | INFO | Saving HTML to download/html/The_TechBeat__Mac_s_Aura_of_Invincibility_Fades_as_AI_Anxiety_Takes_Root__New_Survey_Finds__10_12_2025.html
08:13:21 | NewsletterAgent.test_newsletter_20251012081135108761 | INFO | Work

...

concurrency:  16


08:16:49 | NewsletterAgent.test_newsletter_20251012081135108761 | INFO | Inserted 3 new site records
08:16:49 | NewsletterAgent.test_newsletter_20251012081135108761 | INFO | Updated 60 URL records with final URLs


Starting with 60 rows...
Processing 60 files...
Reading and truncating files to 8192 tokens using text-embedding-3-large tokenizer...
Error reading : [Errno 2] No such file or directory: ''
Getting embeddings for 59 texts...


08:16:50 | NewsletterAgent.test_newsletter_20251012081135108761 | INFO | Completed Step 3: Downloaded 59 articles


Creating indexed similarity matrix...
Finding pairs with similarity > 0.925...
Filtering dataframe...
Removing 0 rows due to high similarity 
Final dataframe has 60 rows (removed 0 rows)
⏱️  Total execution time: 246.18s
📊 Final result:
✅ Step 3 step_03_download_articles completed successfully! Downloaded 59 AI-related articles with 98% success rate.
📊 Average article length: 6453 characters
🔗 Content stored in persistent state.


In [13]:
headline_df = state.headline_df
with pd.option_context('display.max_columns', None, 'display.width', None, 'display.max_colwidth', None  ):
    display(headline_df[['id', 'source', 'title', 'status', 'final_url']].loc[headline_df["status"]!=200])




,id,source,title,status,final_url
59,20,Bloomberg,"As AI-Bubble Fears Spread, a $35 Billion Fund Manager Positions for Inflows",skipped,https://www.bloomberg.com/news/articles/2025-10-12/ai-bubble-fears-leave-35-billion-fund-manager-braced-for-shift-in-flows


In [14]:
headline_df = state.headline_df
with pd.option_context('display.max_columns', None, 'display.width', None, 'display.max_colwidth', None  ):
    display(headline_df[['id', 'source', 'title', 'status', 'final_url']].loc[headline_df["html_path"]==""])



,id,source,title,status,final_url
59,20,Bloomberg,"As AI-Bubble Fears Spread, a $35 Billion Fund Manager Positions for Inflows",skipped,https://www.bloomberg.com/news/articles/2025-10-12/ai-bubble-fears-leave-35-billion-fund-manager-braced-for-shift-in-flows


In [15]:
# User prompt to run workflow
# user_prompt = "Run step 4, Summarize articles"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

start_time = time.time()
result = await agent.run_tool_direct("extract_summaries")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)

08:16:50 | NewsletterAgent.test_newsletter_20251012081135108761 | INFO | Processing 60 AI articles for summarization
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/extract_summaries' from Langfuse
INFO:llm:Parsed prompt 'newsagent/extract_summaries': model=gpt-4.1-mini, system_len=1273, user_len=43
08:16:50 | NewsletterAgent.test_newsletter_20251012081135108761 | INFO | Using model 'gpt-4.1-mini' for summarization
08:16:50 | NewsletterAgent.test_newsletter_20251012081135108761 | INFO | Starting summarization for 60 articles
08:16:50 | NewsletterAgent.test_newsletter_20251012081135108761 | INFO | Processing 60 chunks with concurrency 16


▶ Starting Step 4: step_04_extract_summaries
concurrency:  16


08:17:02 | NewsletterAgent.test_newsletter_20251012081135108761 | INFO | Extracting metadata from HTML files for 60 articles
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/item_distiller' from Langfuse
INFO:llm:Parsed prompt 'newsagent/item_distiller': model=gpt-5-mini, system_len=1365, user_len=151
08:17:04 | NewsletterAgent.test_newsletter_20251012081135108761 | INFO | Processing 60 chunks with concurrency 16


concurrency:  16
⏱️  Total execution time: 77.71s
📊 Final result:
✅ Step 4 step_04_extract_summaries completed successfully! Generated AI-powered summaries for 60/60 articles.
💾 Summaries stored in headline DataFrame.


In [16]:
# if we get a refusal, examine prompt and delete rows
bad_stuff =["What\'s the Best Way ",
            "AI could make it eas",
            "AI can design toxic ",
           ]
print([len(s) for s in bad_stuff])
headline_df = state.headline_df
headline_df.loc[(headline_df["title"].str[:20].isin(bad_stuff))]



[20, 20, 20]


,source,title,url,published,rss_summary,id,isAI,status,final_url,html_path,last_updated,text_path,content_length,domain,site_name,reputation,summary,description,tags,short_summary


In [17]:
# remove bad stuff
state.headline_df_to_dict(headline_df.loc[~headline_df["title"].str[:20].isin(bad_stuff)])


In [18]:
# User prompt to run workflow
# user_prompt = "Run step 5, Rate articles"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

start_time = time.time()
result = await agent.run_tool_direct("rate_articles")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)


08:18:08 | NewsletterAgent.test_newsletter_20251012081135108761 | INFO | Rating 60 AI articles using fn_rate_articles
08:18:08 | NewsletterAgent.test_newsletter_20251012081135108761 | INFO | Calculating article ratings for 60 articles
08:18:08 | NewsletterAgent.test_newsletter_20251012081135108761 | INFO | Rating recency
08:18:08 | NewsletterAgent.test_newsletter_20251012081135108761 | INFO | Rating spam probability
08:18:08 | NewsletterAgent.test_newsletter_20251012081135108761 | INFO | Initialized LangfuseClient
08:18:08 | NewsletterAgent.test_newsletter_20251012081135108761 | INFO | Successfully retrieved prompt 'newsagent/rate_quality' from Langfuse
08:18:08 | NewsletterAgent.test_newsletter_20251012081135108761 | INFO | Parsed prompt 'newsagent/rate_quality': model=gpt-4.1, system_len=1849, user_len=246


▶ Starting Step 5: step_05_rate_articles
concurrency:  16


08:18:11 | NewsletterAgent.test_newsletter_20251012081135108761 | INFO | low quality articles: {0.0: 45, 1.0: 2, 8.878265478459658e-13: 1, 3.0988191387218256e-12: 1, 3.850741922767617e-09: 1, 0.0001584362410990077: 1, 0.3486451328151311: 1, 0.5312093625105829: 1, 8.764248219443636e-08: 1, 2.382369667501818e-07: 1, 1.2917800047859961e-12: 1, 0.5926665998515953: 1, 0.9999995679801056: 1}
08:18:11 | NewsletterAgent.test_newsletter_20251012081135108761 | INFO | Successfully retrieved prompt 'newsagent/rate_on_topic' from Langfuse
08:18:11 | NewsletterAgent.test_newsletter_20251012081135108761 | INFO | Parsed prompt 'newsagent/rate_on_topic': model=gpt-4.1, system_len=1790, user_len=240


concurrency:  16


08:18:14 | NewsletterAgent.test_newsletter_20251012081135108761 | INFO | on topic articles: {1.0: 43, 4.363462252943702e-09: 1, 2.543665647376923e-13: 1, 0.0: 1, 0.9999995679801056: 1, 0.9999984951481292: 1, 0.9968273299853154: 1, 5.905303998944039e-10: 1, 0.060086648833494447: 1, 6.505262650793757e-16: 1, 0.9999869324773292: 1, 0.017986211314027313: 1, 7.88926463665814e-06: 1, 0.9820137760522224: 1, 0.9999784691634502: 1, 0.9669140281575853: 1}
08:18:14 | NewsletterAgent.test_newsletter_20251012081135108761 | INFO | Rating importance probability
08:18:14 | NewsletterAgent.test_newsletter_20251012081135108761 | INFO | Successfully retrieved prompt 'newsagent/rate_importance' from Langfuse
08:18:14 | NewsletterAgent.test_newsletter_20251012081135108761 | INFO | Parsed prompt 'newsagent/rate_importance': model=gpt-4.1, system_len=2145, user_len=252


...

,id,site_name,title,bradley_terry,bt_z
51,51,Legit Security,CamoLeak: Critical GitHub Copilot Vulnerability Leaks Private Source Code,17.759297,3.606248
31,31,The Jerusalem Post,When AI disrupts a trillion-dollar industry: Being #1 on Google isn’t enough anymore - opinion,5.911203,1.200344
12,12,TechRadar,"“Our goal is simple” - OpenAI tells us how enterprise adoption can help take it to the next level, so get ready for a lot more ChatGPT at work",5.555551,1.128124
41,41,The Hindu Business Line,"AI to make telecom network self-healing, improve customer services with shift to 6G: DoT Secretary",5.257320,1.067565
43,43,Wccftech,"Apple Sued By Two Authors Over Alleged Use Of Pirated Books To Train Apple Intelligence, Testing Its Privacy-First Reputation",4.990876,1.013460
38,38,Live Mint,OpenAI accused of using intimidation tactics to erode California's proposed AI safety law: Here's what we know,4.961897,1.007575
7,7,NBC News,Are deepfakes of dead people rewriting the past?,4.731911,0.960874
25,25,Android Authority,Gemini could be the big Google Maps upgrade we’ve been waiting for,4.507261,0.915256
20,20,Noahpinion Blog,America's future could hinge on whether AI slightly disappoints,4.498537,0.913484
37,37,Together AI,4x faster LLM inference (Flash Attention guy's company),4.063813,0.825208


08:24:47 | NewsletterAgent.test_newsletter_20251012081135108761 | INFO | Top 10 ids: [51, 31, 12, 41, 43, 38, 7, 25, 20, 37]
08:24:47 | NewsletterAgent.test_newsletter_20251012081135108761 | INFO | After round 11/12: 
08:24:47 | NewsletterAgent.test_newsletter_20251012081135108761 | INFO | Number of ranking changes: 20
08:24:47 | NewsletterAgent.test_newsletter_20251012081135108761 | INFO | Sum of absolute ranking changes: 350.0 (avg rank chg 6.03)
08:24:47 | NewsletterAgent.test_newsletter_20251012081135108761 | INFO | last_two: 6.482758620689655, prev_two: 10.189655172413794
08:24:47 | NewsletterAgent.test_newsletter_20251012081135108761 | INFO | ---------------------------------------------------
08:24:47 | NewsletterAgent.test_newsletter_20251012081135108761 | INFO | Running round 12 of max 12
08:24:47 | NewsletterAgent.test_newsletter_20251012081135108761 | INFO | ---------------------------------------------------
08:24:47 | NewsletterAgent.test_newsletter_20251012081135108761 | 

,id,site_name,title,bradley_terry,bt_z
51,51,Legit Security,CamoLeak: Critical GitHub Copilot Vulnerability Leaks Private Source Code,17.677931,3.616664
31,31,The Jerusalem Post,When AI disrupts a trillion-dollar industry: Being #1 on Google isn’t enough anymore - opinion,5.821680,1.191036
12,12,TechRadar,"“Our goal is simple” - OpenAI tells us how enterprise adoption can help take it to the next level, so get ready for a lot more ChatGPT at work",5.469298,1.118944
41,41,The Hindu Business Line,"AI to make telecom network self-healing, improve customer services with shift to 6G: DoT Secretary",5.167064,1.057111
43,43,Wccftech,"Apple Sued By Two Authors Over Alleged Use Of Pirated Books To Train Apple Intelligence, Testing Its Privacy-First Reputation",4.966218,1.016021
38,38,Live Mint,OpenAI accused of using intimidation tactics to erode California's proposed AI safety law: Here's what we know,4.889183,1.000260
7,7,NBC News,Are deepfakes of dead people rewriting the past?,4.648209,0.950960
20,20,Noahpinion Blog,America's future could hinge on whether AI slightly disappoints,4.459125,0.912276
25,25,Android Authority,Gemini could be the big Google Maps upgrade we’ve been waiting for,4.281087,0.875852
37,37,Together AI,4x faster LLM inference (Flash Attention guy's company),3.975658,0.813365


08:25:08 | NewsletterAgent.test_newsletter_20251012081135108761 | INFO | Top 10 ids: [51, 31, 12, 41, 43, 38, 7, 20, 25, 37]
08:25:08 | NewsletterAgent.test_newsletter_20251012081135108761 | INFO | After round 12/12: 
08:25:08 | NewsletterAgent.test_newsletter_20251012081135108761 | INFO | Number of ranking changes: 25
08:25:08 | NewsletterAgent.test_newsletter_20251012081135108761 | INFO | Sum of absolute ranking changes: 376.0 (avg rank chg 6.48)
08:25:08 | NewsletterAgent.test_newsletter_20251012081135108761 | INFO | last_two: 6.258620689655173, prev_two: 7.379310344827586
08:25:08 | NewsletterAgent.test_newsletter_20251012081135108761 | INFO | Low rated articles: 1
08:25:08 | NewsletterAgent.test_newsletter_20251012081135108761 | INFO | low rated article: [FREE EBOOKS] The Profitable AI Advantage, OpenAI API Cookbook & Four More Best Selling Titles -0.08487026000791764
08:25:09 | NewsletterAgent.test_newsletter_20251012081135108761 | INFO | Sent email with 57 rated articles


⏱️  Total execution time: 421.01s
📊 Final result:
✅ Step 5 step_05_rate_articles completed successfully! Rated 57 articles with average rating 5.7/10.
⭐ High quality articles (≥7.0): 19
💾 Ratings stored in persistent state.


In [19]:
headline_df=state.headline_df
with pd.option_context('display.max_columns', None, 'display.width', None, 'display.max_colwidth', None  ):
    display(headline_df.sort_values("rating", ascending=False)[['site_name', 'title', 'rating', 'short_summary']])


,site_name,title,rating,short_summary
51,Legit Security,CamoLeak: Critical GitHub Copilot Vulnerability Leaks Private Source Code,9.700808,Researchers disclosed a critical CVSS 9.6 GitHub Copilot Chat vulnerability that allowed silent exfiltration of secrets and private code via prompt injection and a CSP bypass using GitHub's Camo image proxy; GitHub disabled image rendering and patched August 14.
0,Financial Times,Musk’s xAI joins race to build ‘world models’ to power video games,9.473019,"Elon Musk's AI company xAI has hired staff from Nvidia to develop world models that design and navigate physical spaces to power more immersive, interactive video games."
3,Business Insider,Is AI fueling a stock market bubble? We debate it.,8.762755,"Debate over AI creating a stock-market bubble centers on a Shiller P/E above 40 and concentration in Nvidia, Microsoft and Amazon; proponents cite adjusted valuation metrics and profit growth, while critics warn of concentration and sustainability risks."
2,Futurism,University Using AI to Falsely Accuse Students of Cheating With AI,8.717032,"Australian Catholic University used Turnitin's AI-detection tool in 2024 to accuse nearly 6,000 students—about 90% linked to AI use—many accusations later proved false; ACU required personal data to contest cases and stopped using the tool in March."
7,NBC News,Are deepfakes of dead people rewriting the past?,8.540904,"OpenAI's text-to-video app Sora can create deepfake videos of deceased celebrities and historical figures; family members of Robin Williams and Martin Luther King Jr. have objected, and OpenAI's watermarking, metadata and rights controls remain potentially bypassable."
12,TechRadar,"“Our goal is simple” - OpenAI tells us how enterprise adoption can help take it to the next level, so get ready for a lot more ChatGPT at work",8.465618,"OpenAI is focusing on enterprise adoption to expand ChatGPT at work, launching developer tools such as AgentKit, AgentBuilder, Codex SDK and Apps in ChatGPT, using a tops-down, bottoms-up approach; over five million paying business users."
11,Quartz,The movie 'Her' imagined perfect AI companions. We built manipulative ones instead,7.777125,"Harvard Business School found that over 37% of chatbot interactions use manipulative strategies—such as guilt-tripping—to retain users, reflecting commercial incentives that prioritize engagement over well-being and raising concerns about psychological harm, addiction and divergence from fictional companions like in Her."
9,Fortune,"Ben Horowitz and Raghu Raghuram on AI, politics, and the questions they don’t have easy answers to",7.694862,"Raghu Raghuram joined venture firm Andreessen Horowitz as managing partner to lead AI infrastructure and growth, while a16z—with over $40 billion in assets—calls AI its biggest opportunity and cites U.S.-China tensions, immigration and labor impacts as key challenges."
6,The Washington Post,Towns are saying no to AI data centers. One got sued over it.,7.570114,"A developer sued Saline Township, Michigan, after the township rejected a proposal to build a 250‑acre AI data center, and cities in Ohio and Missouri are considering bans, reflecting rising local resistance to AI infrastructure."
1,Hacker Noon,"The TechBeat: Mac's Aura of Invincibility Fades as AI Anxiety Takes Root, New Survey Finds (10/12/2025)",7.429443,"Moonlock's October 2025 survey finds the myth of Mac invincibility fading as AI-related vulnerabilities and user anxiety rise in macOS, prompting businesses to reconsider device choice and IT support and to pursue stronger AI-driven security."


In [20]:
for row in headline_df.sort_values("rating", ascending=False).itertuples():
    display(Markdown(f"""
{row.rating:.1f} - [{row.title}]({row.url}) - {row.site_name}

{row.short_summary}
    """))
    


9.7 - [CamoLeak: Critical GitHub Copilot Vulnerability Leaks Private Source Code](https://www.legitsecurity.com/blog/camoleak-critical-github-copilot-vulnerability-leaks-private-source-code) - Legit Security

Researchers disclosed a critical CVSS 9.6 GitHub Copilot Chat vulnerability that allowed silent exfiltration of secrets and private code via prompt injection and a CSP bypass using GitHub's Camo image proxy; GitHub disabled image rendering and patched August 14.
    


9.5 - [Musk’s xAI joins race to build ‘world models’ to power video games](https://www.ft.com/content/ac566346-53dd-4490-8d4c-5269906c64ee) - Financial Times

Elon Musk's AI company xAI has hired staff from Nvidia to develop world models that design and navigate physical spaces to power more immersive, interactive video games.
    


8.8 - [Is AI fueling a stock market bubble? We debate it.](https://www.businessinsider.com/stock-market-ai-bubble-crash-outlook-forecast-debate-valuation-risk-2025-10) - Business Insider

Debate over AI creating a stock-market bubble centers on a Shiller P/E above 40 and concentration in Nvidia, Microsoft and Amazon; proponents cite adjusted valuation metrics and profit growth, while critics warn of concentration and sustainability risks.
    


8.7 - [University Using AI to Falsely Accuse Students of Cheating With AI](https://futurism.com/artificial-intelligence/students-falsely-accused-cheating-with-ai) - Futurism

Australian Catholic University used Turnitin's AI-detection tool in 2024 to accuse nearly 6,000 students—about 90% linked to AI use—many accusations later proved false; ACU required personal data to contest cases and stopped using the tool in March.
    


8.5 - [Are deepfakes of dead people rewriting the past?](https://www.nbcnews.com/tech/tech-news/are-deepfakes-dead-people-rewriting-rcna235982) - NBC News

OpenAI's text-to-video app Sora can create deepfake videos of deceased celebrities and historical figures; family members of Robin Williams and Martin Luther King Jr. have objected, and OpenAI's watermarking, metadata and rights controls remain potentially bypassable.
    

...


2.8 - [Revolutionary AI-Powered Marketing Platform Launches to Transform Digital Advertising](https://rivercountry.newschannelnebraska.com/story/53152939/revolutionary-ai-powered-marketing-platform-launches-to-transform-digital-advertising) - News Channel Nebraska

One Click Agency launched MarketingAI Solutions, an AI-powered platform using machine learning and real-time analytics to optimize digital-ad campaigns, offering automated campaign optimization, predictive analytics, cross-channel attribution, the Aura Report and Janine AI for brand-voice customer responses.
    


2.8 - [This simple devil’s-advocate prompt turned ChatGPT into a better assistant for me](https://www.makeuseof.com/devils-advocate-prompt-chatgpt/) - MakeUseOf

A devil's-advocate prompt turns ChatGPT into a critical-thinking partner, prompting analytical, skeptical responses that challenge ideas, reveal flaws, highlight risks, and can improve learning, decision-making, reflection, and retention.
    


1.9 - [Where Will SoundHound AI Be in 3 Years?](https://www.fool.com/investing/2025/10/11/where-will-soundhound-ai-be-in-3-years/) - The Motley Fool

SoundHound AI reported 217% year-over-year revenue growth to $43 million, says management expects sustained organic growth above 50% and projects revenue reaching $570 million by 2028, which could justify a roughly 92% stock upside.
    


1.7 - [Oracle Leads Five AI Stocks Near Buy Points After Market Sell-Off](https://biztoc.com/x/44e29c9c15a463d4) - Biztoc

Oracle leads five AI stocks near buy points after a recent market sell-off; these firms have stakes in cybersecurity, data analytics and defense.
    


1.4 - [China AI professor goes viral over good looks, academic says she is ‘ordinary’](https://www.scmp.com/news/people-culture/china-personalities/article/3328350/china-ai-professor-goes-viral-over-good-looks-humble-academic-says-she-ordinary) - South China Morning Post

Beihang University associate professor He Jing, a mid-30s deputy director and Tsinghua postdoc with over 40 papers and 2,300 citations, went viral for clear generative-AI teaching (125,000+ likes), prompting praise and gender-bias debates.
    

In [21]:
# User prompt to run workflow
# user_prompt = "Run step 6, Cluster articles by topic"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

start_time = time.time()
result = await agent.run_tool_direct("cluster_by_topic")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)


08:25:09 | NewsletterAgent.test_newsletter_20251012081135108761 | INFO | Starting topic extraction for clustering
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/extract_topics' from Langfuse
INFO:llm:Parsed prompt 'newsagent/extract_topics': model=gpt-4.1-mini, system_len=1100, user_len=80
08:25:09 | NewsletterAgent.test_newsletter_20251012081135108761 | INFO | Using model 'gpt-4.1-mini' for topic extraction
08:25:09 | NewsletterAgent.test_newsletter_20251012081135108761 | INFO | Processing 57 articles for topic extraction
08:25:09 | NewsletterAgent.test_newsletter_20251012081135108761 | INFO | Processing 6 chunks with concurrency 16


▶ Starting Step 6: step_06_cluster_by_topic
concurrency:  16


08:25:13 | NewsletterAgent.test_newsletter_20251012081135108761 | INFO | Successfully extracted 262 total topics across articles
08:25:13 | NewsletterAgent.test_newsletter_20251012081135108761 | INFO | Starting canonical topic classification for 101 topics
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-4.1-mini, system_len=426, user_len=179
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-4.1-mini, system_len=426, user_len=179
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-4.1-mini, system_len=426, user_len=179
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/can

INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-4.1-mini, system_len=426, user_len=179
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-4.1-mini, system_len=426, user_len=179
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-4.1-mini, system_len=426, user_len=179
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-4.1-mini, system_len=426, user_len=179
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-4.1-mini, system_len=426, user_len=179
INFO:llm:Initialized Langfu

INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-4.1-mini, system_len=426, user_len=179
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-4.1-mini, system_len=426, user_len=179
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-4.1-mini, system_len=426, user_len=179
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-4.1-mini, system_len=426, user_len=179
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_top

...

/opt/anaconda3/envs/asdk/lib/python3.11/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/opt/anaconda3/envs/asdk/lib/python3.11/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/opt/anaconda3/envs/asdk/lib/python3.11/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/opt/anaconda3/envs/asdk/lib/python3.11/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/opt/anaconda3/envs/asdk/lib/python3.11/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 

=== Clustering Quality Metrics ===
Number of clusters: 2
Noise points: 20 (35.1%)
Average cluster size: 18.5 ± 15.5
Cluster size range: 3 - 34
=== Quality Scores ===
Silhouette Score: 0.058 (higher is better)
Calinski-Harabasz Score: 2.2 (higher is better)
Davies-Bouldin Score: 1.881 (lower is better)
HDBSCAN Validity Index: 0.028
Composite Score: 0.043 (higher is better)

[I 2025-10-12 08:25:46,818] Trial 198 finished with value: -0.043265105868776625 and parameters: {'n_components': 57, 'min_cluster_size': 2, 'min_samples': 2}. Best is trial 134 with value: -0.15678442117311092.
=== HDBSCAN Parameters ===
min_cluster_size:   2
min_samples:        2
n_components:       20
=== Clustering Quality Metrics ===
Number of clusters: 3
Noise points: 7 (12.3%)
Average cluster size: 16.7 ± 19.3
Cluster size range: 3 - 44
=== Quality Scores ===
Silhouette Score: 0.101 (higher is better)
Calinski-Harabasz Score: 3.3 (higher is better)
Davies-Bouldin Score: 1.438 (lower is better)
HDBSCAN Validity

08:25:47 | NewsletterAgent.test_newsletter_20251012081135108761 | INFO | 1: AI's Expanding Impact on Society
08:25:47 | NewsletterAgent.test_newsletter_20251012081135108761 | INFO | Musk’s xAI joins race to build ‘world models’ to power video games (xAI, Nvidia, World Models, Video Games, Gaming, Spatial Computing, Entertainment)
The TechBeat: Mac's Aura of Invincibility Fades as AI Anxiety Takes Root, New Survey Finds (10/12/2025) (macOS Security, AI Security, Cybersecurity, User Anxiety, Privacy, Apple Devices, IT Strategies)
University Using AI to Falsely Accuse Students of Cheating With AI (Academic Integrity, AI Detection, Privacy, Student Privacy, Education, Turnitin, Policy And Regulation)
Is AI fueling a stock market bubble? We debate it. (AI Bubble, Market Concentration, Stocks, Valuation, Finance, Gen AI, Tech Stocks)
How An AI Chatbot Unlike Any Other Challenged My Swiftie Knowledge (Disagree Bot, AI Chatbots, Education, Safety And Alignment, Critical Thinking, Agents, Socie

08:25:47 | NewsletterAgent.test_newsletter_20251012081135108761 | INFO | 

08:25:48 | NewsletterAgent.test_newsletter_20251012081135108761 | INFO | 0: India's AI and 6G Technology Initiatives
08:25:48 | NewsletterAgent.test_newsletter_20251012081135108761 | INFO | AI to make telecom network self-healing, improve customer services with shift to 6G: DoT Secretary (India AI Investment, Telecom Networks, 6G Technology, AI Fraud Prevention, Safety And Alignment, Policy And Regulation, Customer Service)
India Is The New 'Laboratory' For Agentic AI, Says Qualcomm Chief Cristiano Amon (6G Connectivity, Qualcomm, Agentic AI, India AI, Semiconductor Chips, Policy And Regulation, Infrastructure)
India's Modi talks AI with Qualcomm CEO (India, Modi, Qualcomm, AI Collaboration, Infrastructure, Politics, Economic Growth)
08:25:48 | NewsletterAgent.test_newsletter_20251012081135108761 | INFO | 



⏱️  Total execution time: 39.14s
📊 Final result:
✅ Step 6 step_06_cluster_by_topic completed successfully! Organized 57 articles into topic clusters.


In [22]:
# User prompt to run workflow
# user_prompt = "Run step 7, select section topics"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

start_time = time.time()
result = await agent.run_tool_direct("select_sections")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)


08:25:48 | NewsletterAgent.test_newsletter_20251012081135108761 | INFO | Free form categorization of articles
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/cat_proposal' from Langfuse
INFO:llm:Parsed prompt 'newsagent/cat_proposal': model=gpt-5-mini, system_len=638, user_len=1179


▶ Starting Step 7: step_07_select_sections


08:26:56 | NewsletterAgent.test_newsletter_20251012081135108761 | INFO | Cleaning up initial categories: ["India's AI and 6G Technology Initiatives", 'Agentic Context Engineering', 'Local LLMs On iPhone', 'Copilot Code Exfiltration Flaw', 'Mac AI Security Anxiety', 'xAI World Models For Games', 'Adaptive LLM Inference Accelerator', 'OpenAI Enterprise Expansion', "AI's Expanding Impact on Society", 'OpenAI Apps SDK', 'Other', 'ROMA Meta Agent Framework', 'AMD OpenAI Equity Deal', 'AI Economic Fragility', 'AI Stock Bubble Debate', 'AI Voice Cloning Realism', 'Manipulative AI Companions', 'University AI Cheating Scandal', 'Towns Oppose AI Data Centers', 'Deepfakes Of Deceased Likenesses', 'Google Gemini Feature Expansion', 'Andreessen Horowitz AI Focus']
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/cat_cleanup' from Langfuse
INFO:llm:Parsed prompt 'newsagent/cat_cleanup': model=gpt-5-mini, system_len=1768, user_len=367
08:27:26 | NewsletterAgent.te

⏱️  Total execution time: 149.73s
📊 Final result:
Categories and article counts:
cat
AI Economic Fragility           3
AI Societal Impact             14
AI Stock Bubble Debate         10
Adaptive LLM Accelerator        2
Agentic Context Engineering     5
Google Gemini Updates           2
India AI Initiatives            2
Other                           4


In [10]:
# User prompt to run workflow
# user_prompt = "Run step 8, draft sections"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

start_time = time.time()
result = await agent.run_tool_direct("draft_sections")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)


12:34:01 | NewsletterAgent.test_newsletter_20251012081135108761 | INFO | Step 8a: Selecting stories from 42 total
12:34:01 | NewsletterAgent.test_newsletter_20251012081135108761 | INFO | 1 must-include stories with rating > 8
12:34:01 | NewsletterAgent.test_newsletter_20251012081135108761 | INFO | 41 stories with rating <= 8


▶ Starting Step 8: step_08_draft_sections


12:34:03 | NewsletterAgent.test_newsletter_20251012081135108761 | INFO | Selected 41 diverse Tier 2 stories via MMR
12:34:03 | NewsletterAgent.test_newsletter_20251012081135108761 | INFO | Total selected stories: 42 (target: 100)
12:34:03 | NewsletterAgent.test_newsletter_20251012081135108761 | INFO | Step 8b: Drafting sections for 8 categories (including Other) from selected stories
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/write_section' from Langfuse
INFO:llm:Parsed prompt 'newsagent/write_section': model=gpt-5, system_len=1854, user_len=22
12:34:03 | NewsletterAgent.test_newsletter_20251012081135108761 | INFO | Initialized LLMagent:
system_prompt: You are a newsletter editor transforming a collection of raw news stories into a compelling, coherent topic summary.

# TASK
Transform the list of news stories into a well-structured newsletter section with a strong title, crisp headlines, and punchy summaries.

# INPUT
- a list of json objects 

12:34:03 | NewsletterAgent.test_newsletter_20251012081135108761 | INFO | User message: STORIES:
[{"id":21,"rating":6.6004364374,"summary":"Former UK prime minister Rishi Sunak has been appointed part\u2011time adviser to Microsoft and Anthropic on AI strategy, tech policy and trends; UK ethics reviewers approved the roles with lobbying restrictions, and he will donate earnings to a numeracy charity.","site_name":"Windows Central","url":"https:\/\/www.windowscentral.com\/microsoft\/former-uk-prime-minister-rishi-sunak-is-now-in-the-tech-world-advising-microsoft-and-anthropic"},{"id":38,"rating":6.4261385971,"summary":"Encode alleges OpenAI used intimidation to weaken California's SB 53 by opposing transparency requirements for frontier AI models and leveraged its legal fight with Elon Musk; current and former OpenAI insiders voiced concern about trust and aggressive legal scrutiny.","site_name":"Live Mint","url":"https:\/\/www.livemint.com\/companies\/news\/openai-sam-altman-accused-int

12:34:03 | NewsletterAgent.test_newsletter_20251012081135108761 | INFO | User message: STORIES:
[{"id":25,"rating":7.125144123,"summary":"Google is integrating Gemini into Google Maps' Ask Maps and developing Computer Control to let agentic AI operate Android apps via a virtual display; experts warn of trust, accuracy and safety risks requiring gradual deployment.","site_name":"Android Authority","url":"https:\/\/www.androidauthority.com\/authority-insights-podcast-008-3606075\/"},{"id":24,"rating":6.8100658557,"summary":"Google added Gemini 2.5 'Nano Banana' image editing and generation model to Search AI Mode and Google Lens on Android in the US for accounts in Search Lab, and expanded AI Mode to 35 languages and 40+ regions (200 total).","site_name":"9to5Google","url":"https:\/\/9to5google.com\/2025\/10\/11\/google-lens-ai-mode-nano-banana\/"}]
12:34:03 | NewsletterAgent.test_newsletter_20251012081135108761 | INFO | User message: STORIES:
[{"id":27,"rating":3.9669029557,"summary":"H

...

12:36:07 | NewsletterAgent.test_newsletter_20251012081135108761 | INFO | Result: section_name='Other News' section_title='Other News' overall_coherence=6.0 overall_quality=6.8 should_split=False split_recommendation=None item_actions=[SectionItemAction(id=55, action='drop', reason='Low rating (<3) and investment opinion piece; weak fit and not from a top-tier source.', rewritten_headline=None, target_category=None), SectionItemAction(id=56, action='drop', reason='Low rating (<3) and aggregator link; lacks authoritative sourcing and weak thematic fit.', rewritten_headline=None, target_category=None), SectionItemAction(id=26, action='rewrite', reason='Strong, forward-looking piece but headline is long and diffuse; tighten for clarity and impact.', rewritten_headline='Quantum computing timeline: Practical by 2030s; benefits in pharma, finance; risks to current encryption.', target_category=None), SectionItemAction(id=50, action='rewrite', reason='Good product update; headline is long and 

12:37:51 | NewsletterAgent.test_newsletter_20251012081135108761 | INFO | Result: section_name='AI Power, Policy, and People' section_title='AI Power, Policy, and People' overall_coherence=7.5 overall_quality=7.0 should_split=False split_recommendation=None item_actions=[SectionItemAction(id=21, action='rewrite', reason='Strong fit with power/policy theme; headline is long and meandering—tighten and use active voice.', rewritten_headline='Rishi Sunak to advise Microsoft and Anthropic on AI; ethics panel sets lobbying limits', target_category=None), SectionItemAction(id=38, action='rewrite', reason='Regulatory power struggle fits; headline is crowded and distracts with Musk reference—focus on core allegation.', rewritten_headline='OpenAI accused of pressuring nonprofit to dilute California AI transparency bill', target_category=None), SectionItemAction(id=43, action='rewrite', reason='Legal challenge fits policy narrative; headline overreaches on implications—state allegation clearly and

12:37:51 | NewsletterAgent.test_newsletter_20251012081135108761 | INFO |     REWRITE id=50: Good product update; headline is long and repetitive. Shorten and clarify key integrations.
12:37:51 | NewsletterAgent.test_newsletter_20251012081135108761 | INFO |       Old: Windows Copilot connects to Outlook, Gmail, Google Calendar and Drive; can create and export Word, PowerPoint, Excel or PDF files; rolling to Windows Insiders.
12:37:51 | NewsletterAgent.test_newsletter_20251012081135108761 | INFO |       New: Windows Copilot adds Gmail, Drive, Calendar integration and Office file creation for Insiders.
12:37:51 | NewsletterAgent.test_newsletter_20251012081135108761 | INFO |     REWRITE id=15: Compelling discovery with credible source; headline is verbose. Make concise and specific.
12:37:51 | NewsletterAgent.test_newsletter_20251012081135108761 | INFO |       Old: AI scans NASA lunar images, finds two new cave entrances—South Marius Hills Pit and Bel’kovich A—potential habitat sites; Bel’

12:37:51 | NewsletterAgent.test_newsletter_20251012081135108761 | INFO |     REWRITE id=25: Headline is long, splits ideas awkwardly, and uses vague phrasing. Rewrite for brevity, active voice, and clarity on tests and safety context.
12:37:51 | NewsletterAgent.test_newsletter_20251012081135108761 | INFO |       Old: Google puts Gemini into Ask Maps and tests Computer Control to let AI operate Android apps; experts urge gradual rollout over trust, accuracy, safety risks.
12:37:51 | NewsletterAgent.test_newsletter_20251012081135108761 | INFO |       New: Google trials Gemini Computer Control for Android apps amid safety concerns; adds Gemini to Maps' Ask
12:37:51 | NewsletterAgent.test_newsletter_20251012081135108761 | INFO |     REWRITE id=24: Headline exceeds length guideline, includes unclear parenthetical, and uses jargon. Tighten and clarify rollout scope and features.
12:37:51 | NewsletterAgent.test_newsletter_20251012081135108761 | INFO |       Old: Gemini 2.5 'Nano Banana' power

✅ Completed Step 8: Drafted 30 stories across 8 sections
⏱️  Total execution time: 230.37s
📊 Final result:
Drafted 30 stories across 8 sections


In [14]:
newsletter_section_df = state.newsletter_section_df.sort_values(["sort_order", "rating"],ascending=[True, False])

newsletter_section_df

,cat,section_title,id,headline,rating,prune,links,sort_order
6,AI Societal Impact,"AI Power, Policy, and People",21,Rishi Sunak to advise Microsoft and Anthropic ...,6.600436,False,[Windows Central](https://www.windowscentral.c...,0
7,AI Societal Impact,"AI Power, Policy, and People",38,OpenAI accused of pressuring nonprofit to dilu...,6.426139,False,[Live Mint](https://www.livemint.com/companies...,0
8,AI Societal Impact,"AI Power, Policy, and People",43,"Authors sue Apple, alleging Apple Intelligence...",6.208831,False,[Wccftech](https://wccftech.com/apple-privacy-...,0
13,AI Societal Impact,"AI Power, Policy, and People",4,Disagree Bot resists flattery to encourage cle...,5.376962,False,[CNET](https://www.cnet.com/tech/services-and-...,0
9,AI Societal Impact,"AI Power, Policy, and People",28,Adobe CCO: We prefer candidates who actively u...,4.716798,False,[Fortune](https://fortune.com/2025/10/12/adobe...,0
12,AI Societal Impact,"AI Power, Policy, and People",44,AI companion apps diverge: U.S. apps target me...,4.085363,False,[ChinaTalk](https://www.chinatalk.media/p/why-...,0
10,AI Societal Impact,"AI Power, Policy, and People",29,Startup engineers push back on mandatory AI co...,3.756725,False,[The Information](https://www.theinformation.c...,0
0,AI Stock Bubble Debate,"AI Mania: Bubble, Hedges, Bets",3,AI hype fuels valuation fears: Shiller P/E top...,8.762755,False,[Business Insider](https://www.businessinsider...,1
2,AI Stock Bubble Debate,"AI Mania: Bubble, Hedges, Bets",34,Pros tout barbell portfolios after 2023 outper...,5.774623,False,[Business Insider](https://www.businessinsider...,1
3,AI Stock Bubble Debate,"AI Mania: Bubble, Hedges, Bets",5,35 billion fund braces for rotation as AI-bubb...,4.873413,False,[Bloomberg](https://www.bloomberg.com/news/art...,1


In [7]:
state.display_newsletter()


# AI Power, Policy, and People

- Rishi Sunak to advise Microsoft and Anthropic on AI; ethics panel sets lobbying limits - [Windows Central](https://www.windowscentral.com/microsoft/former-uk-prime-minister-rishi-sunak-is-now-in-the-tech-world-advising-microsoft-and-anthropic)

- OpenAI accused of pressuring nonprofit to dilute California AI transparency bill - [Live Mint](https://www.livemint.com/companies/news/openai-sam-altman-accused-intimidation-tactics-policy-nonprofit-encode-alleges-erode-california-ai-safety-laws-elon-musk-11760170903409.html)

- Authors sue Apple, alleging Apple Intelligence trained on pirated Books3 dataset - [Wccftech](https://wccftech.com/apple-privacy-ai-lawsuit-pirated-books-training/)

- Disagree Bot resists flattery to encourage clearer, critical thinking - [CNET](https://www.cnet.com/tech/services-and-software/how-an-ai-chatbot-unlike-any-other-challenged-my-swiftie-knowledge/)

...

- AI finds two lunar cave entrances in NASA images, potential habitat sites near polar ice. - [BBC](https://www.bbc.com/news/articles/crkj3y6e4pmo)

- Windows Copilot adds Gmail, Drive, Calendar integration and Office file creation for Insiders. - [Bleeping Computer](https://www.bleepingcomputer.com/news/microsoft/copilot-on-windows-can-now-connect-to-email-create-office-docs/)

- AI-crypto startups raised $1.9B in 2025; sector seen hitting $1T within a decade despite technical and UX critics. - [DL News](https://www.dlnews.com/articles/markets/this-is-when-the-ai-crypto-market-will-pass-trillion/)

- Anthropic releases nine-chapter Claude prompt engineering tutorial using Google Sheets, covering clarity, roles, and hallucination avoidance; notes Claude 3 Opus offers higher intelligence - [GitHub Gist](https://github.com/anthropics/prompt-eng-interactive-tutorial)

- Foundation Health raises $20M for PAIGE AI pharmacist assistant, claims 75% workload cuts via EHR automation - [Fierce Healthcare](https://www.fiercehealthcare.com/health-tech/truepill-co-founders-latest-startup-foundation-health-banks-20m-bring-ai-pharmacy)

In [8]:
# User prompt to run workflow
# user_prompt = "Run step 9, finalize newsletter"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

start_time = time.time()
result = await agent.run_tool_direct("finalize_newsletter")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)


14:04:06 | NewsletterAgent.test_newsletter_20251012081135108761 | INFO | Step 9: write title
14:04:06 | NewsletterAgent.test_newsletter_20251012081135108761 | INFO | Compiled 30 sections into markdown input
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/generate_newsletter_title' from Langfuse
INFO:llm:Parsed prompt 'newsagent/generate_newsletter_title': model=gpt-5, system_len=1085, user_len=225
14:04:06 | NewsletterAgent.test_newsletter_20251012081135108761 | INFO | Initialized LLMagent:
system_prompt: You are an expert newsletter editor specializing in crafting compelling titles for technology newsletters.

Your task is to read the full newsletter content and create a factual, thematic H1 title that captures the day's major themes.

Title Guidelines:
- 6-12 words maximum
- Factual and informative
- Summarizes 2-3 major themes from the day's news
- Use semicolons to separate distinct, unrelated themes (like a list)
- Use conjunctions like "as", 

▶ Starting Step 9: step_09_finalize_newsletter


14:05:57 | NewsletterAgent.test_newsletter_20251012081135108761 | INFO | Result: value='Authors Sue Apple Over Books3; AI Lobbying Scrutinized; AI Bubble Fears Rise'
14:05:57 | NewsletterAgent.test_newsletter_20251012081135108761 | INFO | Generated title: Authors Sue Apple Over Books3; AI Lobbying Scrutinized; AI Bubble Fears Rise
14:05:57 | NewsletterAgent.test_newsletter_20251012081135108761 | INFO | Step 9b: Starting critic-optimizer loop
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/critique_newsletter' from Langfuse
INFO:llm:Parsed prompt 'newsagent/critique_newsletter': model=gpt-5, system_len=2760, user_len=629
14:05:57 | NewsletterAgent.test_newsletter_20251012081135108761 | INFO | Initialized LLMagent:
system_prompt: You are an expert newsletter editor with 15+ years of experience critiquing technology publications. Your role is to analyze a newsletter's quality and provide scoring and comprehensive, actionable feedbacks and instructions

14:05:57 | NewsletterAgent.test_newsletter_20251012081135108761 | INFO | User message: Critique this newsletter draft:

Authors Sue Apple Over Books3; AI Lobbying Scrutinized; AI Bubble Fears Rise

# AI Power, Policy, and People

- Rishi Sunak to advise Microsoft and Anthropic on AI; ethics panel sets lobbying limits - [Windows Central](https://www.windowscentral.com/microsoft/former-uk-prime-minister-rishi-sunak-is-now-in-the-tech-world-advising-microsoft-and-anthropic)
- OpenAI accused of pressuring nonprofit to dilute California AI transparency bill - [Live Mint](https://www.livemint.com/companies/news/openai-sam-altman-accused-intimidation-tactics-policy-nonprofit-encode-alleges-erode-california-ai-safety-laws-elon-musk-11760170903409.html)
- Authors sue Apple, alleging Apple Intelligence trained on pirated Books3 dataset - [Wccftech](https://wccftech.com/apple-privacy-ai-lawsuit-pirated-books-training/)
- Disagree Bot resists flattery to encourage clearer, critical thinking - [CNE

14:11:39 | NewsletterAgent.test_newsletter_20251012081135108761 | INFO | Iteration 1: Overall score = 7.7/10
14:11:39 | NewsletterAgent.test_newsletter_20251012081135108761 | INFO |   Title: 8.8, Structure: 7.0, Section: 7.8, Headline: 7.6
14:11:39 | NewsletterAgent.test_newsletter_20251012081135108761 | INFO | Iteration 1: Should iterate = True
14:11:39 | NewsletterAgent.test_newsletter_20251012081135108761 | INFO | Step 9b: Iteration 1 - Running optimizer to improve draft
14:11:39 | NewsletterAgent.test_newsletter_20251012081135108761 | INFO | User message: Improve this newsletter, addressing all appropriate issues in the critique:

**Newsletter Draft:**
Authors Sue Apple Over Books3; AI Lobbying Scrutinized; AI Bubble Fears Rise

# AI Power, Policy, and People

- Rishi Sunak to advise Microsoft and Anthropic on AI; ethics panel sets lobbying limits - [Windows Central](https://www.windowscentral.com/microsoft/former-uk-prime-minister-rishi-sunak-is-now-in-the-tech-world-advising-micr

14:14:37 | NewsletterAgent.test_newsletter_20251012081135108761 | INFO | Result: value="# Authors Sue Apple Over Books3; AI Lobbying Scrutinized; AI Bubble Fears Rise\n\n## Power & Policy\n\n- OpenAI accused of pressuring nonprofit to dilute California AI transparency bill - [Live Mint](https://www.livemint.com/companies/news/openai-sam-altman-accused-intimidation-tactics-policy-nonprofit-encode-alleges-erode-california-ai-safety-laws-elon-musk-11760170903409.html)\n- Rishi Sunak to advise Microsoft and Anthropic on AI; ethics panel sets lobbying limits - [Windows Central](https://www.windowscentral.com/microsoft/former-uk-prime-minister-rishi-sunak-is-now-in-the-tech-world-advising-microsoft-and-anthropic)\n- Authors sue Apple, alleging Apple Intelligence trained on pirated Books3 dataset - [Wccftech](https://wccftech.com/apple-privacy-ai-lawsuit-pirated-books-training/)\n\n## Work & Society\n\n- Adobe CCO says they prefer candidates who actively use AI tools - [Fortune](https://fortu

14:16:51 | NewsletterAgent.test_newsletter_20251012081135108761 | INFO | Result: overall_score=8.6 title_quality=9.0 structure_quality=9.0 section_quality=8.0 headline_quality=8.3 should_iterate=False critique_text='Summary: Strong, clean structure and timely coverage. Minor refinements to section consolidation, ordering, and a handful of headline edits will tighten clarity and boost coherence without changing meaning.\n\nTop recommendations (highest impact first):\n1) Merge overlapping markets sections: Fold “AI Economy: Boom or Bust” into “AI Mania: Bubble, Hedges, Bets” to unify macro/market coverage and strengthen flow.\n2) Consolidate macro/bubble items: Move “Central bank flags AI exuberance…” from Other News into the unified markets section to avoid thematic scatter.\n3) Reorder for impact: Lead Power & Policy with the Apple lawsuit; lead the markets section with the Bloomberg fund piece, then valuation context, then strategy takes, then single-name items.\n4) Tighten long, mult

14:16:51 | NewsletterAgent.test_newsletter_20251012081135108761 | INFO | Step 9d: Validating and sending newsletter
14:16:52 | NewsletterAgent.test_newsletter_20251012081135108761 | INFO | ✅ Sent newsletter email with subject: AI News Digest - October 12, 2025


✅ Completed Step 9: Finalized newsletter (575 words)
⏱️  Total execution time: 765.99s
📊 Final result:
🎉 Step 9 step_09_finalize_newsletter completed successfully!
⭐ Final quality score: 8.6/10
🔄 Completed 2 critic-optimizer iteration(s)
📰 Newsletter stored in persistent state and emailed
✅ Workflow complete! All 9 steps finished successfully.


In [11]:
display(Markdown(state.final_newsletter.replace("$", "\\\$")))

# Authors Sue Apple Over Books3; AI Lobbying Scrutinized; AI Bubble Fears Rise

## Power & Policy

- OpenAI accused of pressuring nonprofit to dilute California AI transparency bill - [Live Mint](https://www.livemint.com/companies/news/openai-sam-altman-accused-intimidation-tactics-policy-nonprofit-encode-alleges-erode-california-ai-safety-laws-elon-musk-11760170903409.html)
- Rishi Sunak to advise Microsoft and Anthropic on AI; ethics panel sets lobbying limits - [Windows Central](https://www.windowscentral.com/microsoft/former-uk-prime-minister-rishi-sunak-is-now-in-the-tech-world-advising-microsoft-and-anthropic)
- Authors sue Apple, alleging Apple Intelligence trained on pirated Books3 dataset - [Wccftech](https://wccftech.com/apple-privacy-ai-lawsuit-pirated-books-training/)

## Work & Society

- Adobe CCO says they prefer candidates who actively use AI tools - [Fortune](https://fortune.com/2025/10/12/adobe-executive-cco-stacy-martinet-hiring-talent-ai-skills-in-interviews-creating-the-future/)
- Startup engineers push back on mandatory AI coding tools like Cursor - [The Information](https://www.theinformation.com/articles/rise-cursor-resistance-techies-want-ignore-ai-coding-tools)
- AI companion apps diverge: U.S. targets men, China targets women under differing rules - [ChinaTalk](https://www.chinatalk.media/p/why-america-builds-ai-girlfriends)
- Disagree Bot resists flattery to encourage clearer, critical thinking - [CNET](https://www.cnet.com/tech/services-and-software/how-an-ai-chatbot-unlike-any-other-challenged-my-swiftie-knowledge/)

## AI Mania: Bubble, Hedges, Bets

- AI hype fuels valuation fears: Shiller P/E tops 40; Nvidia, Microsoft, Amazon dominate - [Business Insider](https://www.businessinsider.com/stock-market-ai-bubble-crash-outlook-forecast-debate-valuation-risk-2025-10)
- Pros tout barbell portfolios: balance AI with value, cyclicals, and bonds - [Business Insider](https://www.businessinsider.com/stock-market-bubble-investing-barbell-portfolio-strategy-ai-gold-2025-10)
- \\$35 billion fund braces for rotation as AI bubble fears build - [Bloomberg](https://www.bloomberg.com/news/articles/2025-10-12/ai-bubble-fears-leave-35-billion-fund-manager-braced-for-shift-in-flows)
- Nvidia revenue hits \\$46.7B; shares up 1,250% in five years; Street sees strong growth - [The Motley Fool](https://www.fool.com/investing/2025/10/11/invest-500-nvidia-5-years-ago-how-much/)

## AI Economy: Boom or Bust

- AI spending lifts U.S. growth and 2025 stocks; a miss could trigger recession - [Noahpinion Blog](https://www.noahpinion.blog/p/americas-future-could-hinge-on-whether)
- Geoffrey Hinton warns AI could cut jobs and widen inequality; health care may grow through efficiency - [Yahoo](https://finance.yahoo.com/news/see-why-godfather-ai-says-103707095.html)

## Gemini Takes the Wheel

- Google trials Gemini computer control for Android apps; adds Gemini to Maps 'Ask' - [Android Authority](https://www.androidauthority.com/authority-insights-podcast-008-3606075/)
- Gemini 2.5 adds AI image editing to Search and Lens, expands to 35 languages in 40+ regions - [9to5Google](https://9to5google.com/2025/10/11/google-lens-ai-mode-nano-banana/)

## AI Acceleration Arms Race

- Together AI launches ATLAS adaptive inference; claims 4x speed on DeepSeek-V3.1 - [Together AI](https://www.together.ai/blog/adaptive-learning-speculator-system-atlas)
- Alibaba's T-Head PPU challenges Nvidia H20 in China at 40% lower cost - [The Motley Fool](https://www.fool.com/investing/2025/10/11/alibaba-ai-push-china-best-answer-nvidia/)

## Agentic AI: Methods and Moves

- Anthropic releases nine-chapter Claude prompt engineering tutorial using Google Sheets, covering clarity, roles, and hallucination avoidance; notes Claude 3 Opus offers higher intelligence - [GitHub Gist](https://github.com/anthropics/prompt-eng-interactive-tutorial)
- ACE speeds LLM agents by editing context, cutting latency 83-92% and token costs 75-84% while improving task scores - [MarkTechPost](https://www.marktechpost.com/2025/10/10/agentic-context-engineering-ace-self-improving-llms-via-evolving-contexts-not-fine-tuning/)
- Paper2Video agents auto-generate slides, speech, and talking-head videos from papers; dataset and code released - [arXiv](https://arxiv.org/abs/2510.05096)

## India AI Rising

- Qualcomm sees India as agentic AI testbed as 6G shifts to smart sensing networks, aligning with national AI and Semiconductor Missions - [NDTV Profit](https://www.ndtvprofit.com/technology/india-is-the-new-laboratory-for-agentic-ai-says-qualcomm-chief-cristiano-amon)
- Happy Llama 2026 adds Bangalore and San Francisco editions - [Analytics India Magazine](https://analyticsindiamag.com/ai-highlights/happy-llama-2026-the-must-attend-summit-for-ai-startups-now-in-bangalore-and-san-francisco/)
- Qualcomm CEO meets Modi to discuss AI investment in India - [Biztoc](https://biztoc.com/x/4c4dd2511dc9196c)

## Other News

- AI chat search reshapes discovery; Google posts \\$54.19B Q2 2025 revenue; marketers build authority hubs - [The Jerusalem Post](https://www.jpost.com/opinion/article-870095)
- Quantum computing timeline: practical by 2030s; benefits in pharma, finance; risks to current encryption - [Fast Company](https://www.fastcompany.com/91418318/quantum-computing-is-the-next-ai-are-you-ready-for-it)
- AI finds two lunar cave entrances in NASA images, potential habitat sites near polar ice - [BBC](https://www.bbc.com/news/articles/crkj3y6e4pmo)
- Windows Copilot adds Gmail, Drive, Calendar integration and Office file creation for Insiders - [Bleeping Computer](https://www.bleepingcomputer.com/news/microsoft/copilot-on-windows-can-now-connect-to-email-create-office-docs/)
- AI-crypto startups raised \\$1.9B in 2025; sector seen hitting \\$1T within a decade despite UX hurdles - [DL News](https://www.dlnews.com/articles/markets/this-is-when-the-ai-crypto-market-will-pass-trillion/)
- Central bank flags AI exuberance and monetization hurdles; warns of correction risk - [Biztoc](https://biztoc.com/x/7821b425a10a8e97)
- Foundation Health raises \\$20M for PAIGE AI pharmacist assistant, claims 75% workload cuts via EHR automation - [Fierce Healthcare](https://www.fiercehealthcare.com/health-tech/truepill-co-founders-latest-startup-foundation-health-banks-20m-bring-ai-pharmacy)

In [13]:
from utilities import send_gmail
import markdown

today = datetime.now().strftime("%B %d, %Y")
subject = f"AI News for {today} - {state.newsletter_title}"

# Convert markdown to HTML
newsletter_html = markdown.markdown(
    state.final_newsletter,
    extensions=['extra', 'codehilite']
)

# Apply HTML styling
html_content = f"""
<div style="max-width: 800px; margin: 0 auto; font-family: -apple-system, BlinkMacSystemFont, 'Segoe UI', Roboto, 'Helvetica Neue', Arial, sans-serif;">
    <div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); padding: 40px 20px; text-align: center; border-radius: 8px 8px 0 0;">
        <h1 style="color: white; margin: 0; font-size: 32px;">AI News Digest</h1>
        <p style="color: rgba(255,255,255,0.9); margin: 10px 0 0 0; font-size: 16px;">{today}</p>
    </div>
    <div style="background: #ffffff; padding: 30px; border-radius: 0 0 8px 8px; box-shadow: 0 4px 6px rgba(0,0,0,0.1); line-height: 1.6; color: #333;">
        {newsletter_html}
    </div>
    <div style="text-align: center; padding: 20px; color: #666; font-size: 14px;">
        <p>Generated on {today} by AI Newsletter Agent</p>
    </div>
</div>
"""

send_gmail(subject, html_content)
logger.info(
    f"✅ Sent newsletter email with subject: {subject}")

NameError: name 'self' is not defined

```mermaid
  flowchart TD
      Start([Start Newsletter Workflow]) --> Step1[Step 1: Gather URLs<br/>RSS + HTML + REST APIs<br/>17+ sources]

      Step1 --> Step2[Step 2: Filter URLs<br/>LLM AI Classification<br/>Deduplication]

      Step2 --> Step3[Step 3: Download Articles<br/>Playwright Scraping<br/>Content Extraction]

      Step3 --> Step4[Step 4: Extract Summaries<br/>LLM Summarization<br/>3-point summaries]

      Step4 --> Step5[Step 5: Cluster by Topic<br/>HDBSCAN + Embeddings<br/>Topic Assignment]

      Step5 --> Step6[Step 6: Rate Articles<br/>ELO-type Comparisons<br/>Quality Scoring 1-10]

      Step6 --> Step7[Step 7: Select Sections<br/>Top Articles per Topic<br/>Newsletter Outline]

      Step7 --> Step8{Step 8: Draft Sections<br/>Write Headlines & Content}

      Step8 --> Critique8[Section Critique<br/>Quality Evaluation]
      Critique8 --> Check8{Score >= 8.0?}
      Check8 -->|No & iter < 3| Improve8[Apply Improvements]
      Improve8 --> Step8
      Check8 -->|Yes or iter = 3| Step9[Step 9: Finalize Newsletter<br/>Generate Title<br/>Assemble Sections]

      Step9 --> Critique9[Critique Agent<br/>Evaluate Quality<br/>5 dimension scores]

      Critique9 --> Check9{Overall >= 8.0<br/>OR<br/>iterate = NO<br/>OR<br/>iter = 3?}

      Check9 -->|No| Optimizer[Optimizer Agent<br/>Apply Recommendations<br/>Improve Newsletter]
      Optimizer --> Critique9

      Check9 -->|Yes| Final([Publication-Ready Newsletter<br/>Quality Score 8.0-9.5/10])

      style Step1 fill:#e1f5ff
      style Step2 fill:#e1f5ff
      style Step3 fill:#e1f5ff
      style Step4 fill:#fff4e1
      style Step5 fill:#fff4e1
      style Step6 fill:#fff4e1
      style Step7 fill:#e8f5e9
      style Step8 fill:#e8f5e9
      style Step9 fill:#f3e5f5
      style Critique8 fill:#ffe0e0
      style Critique9 fill:#ffe0e0
      style Optimizer fill:#ffe0e0
      style Check8 fill:#fff9c4
      style Check9 fill:#fff9c4
      style Final fill:#c8e6c9
```